In [1]:
import rioxarray as rioxr
import xarray as xr
import os

In [2]:
filepath = 'data\\raw\\earth_engine_geowiki'
filepath2 = 'data\\raw\\earth_engine_geowiki_2'
#da = xr.open_rasterio(filepath).rename("FEATURES") / 10000
#da = rioxr.open_rasterio(filepath)

In [3]:
base = 'data\\raw\\earth_engine_geowiki\\0-geowiki-landcover-2017_2016-02-07_2017-02-01.tif'
da = rioxr.open_rasterio(base)
band_names = da.attrs['long_name']
keys_toremove = ['VV','VH','total','temperature','elevation','slope']
indices_toremove = []
for idx, name in enumerate(band_names):
    if(any(map(name.__contains__, keys_toremove))):
        indices_toremove.append(idx+1)
indices_toremove2 = [x-1 for x in indices_toremove]
names = list(da.attrs['long_name'])
for index in sorted(indices_toremove2, reverse=True):
    del names[index]
descr = tuple(names)

In [ ]:
for root, dirs, files in os.walk(filepath):
    for file in files:
        da = rioxr.open_rasterio(os.path.join(filepath, file))

        da = da.drop(indices_toremove, dim='band')

        da.attrs['long_name'] = descr

        da.rio.to_raster(os.path.join(filepath2, file))

        del da

In [21]:
from dask.diagnostics import ProgressBar

In [ ]:
filepath = 'data\\raw\\earth_engine_geowiki\\*.tif'
da = rioxr.open_rasterio(filepath)
da.data_vars

In [4]:
filepath = 'data\\raw\\earth_engine_geowiki\\*.tif'
da = xr.open_mfdataset(filepath, parallel = True,engine = 'rasterio')

,Array,Chunk
Bytes,1.14 TiB,5.60 MiB
Shape,"(206, 11419, 66397)","(206, 18, 198)"
Count,12740671 Tasks,4171884 Chunks
Type,float64,numpy.ndarray


In [7]:
with ProgressBar():    
    da = da.drop(indices_toremove, dim='band')
    da.attrs['long_name'] = descr

In [22]:
da2 = xr.open_mfdataset(base, parallel = True,engine = 'rasterio')

In [13]:
da2 = da2.drop_sel(band = indices_toremove)
da2.attrs['long_name'] = descr

In [39]:
da.rio.to_raster(os.path.join(filepath2, '0-geowiki-landcover-2017_2016-02-07_2017-02-01.tif'))

In [35]:
da2.squeeze().rio.to_raster(os.path.join(filepath2, '0-geowiki-landcover-2017_2016-02-07_2017-02-01.tif'))

TooManyDimensions: Only 2D and 3D data arrays supported. Data variable: stack-95f69c5860ab358b33469c318c95bbe4

In [1]:
import pickle
import numpy

In [8]:
with open("2-geowiki-landcover-2017_2016-02-07_2017-02-01.pkl","rb") as f:
    nparray = pickle.load(f)

In [9]:
nparray.labelled_array.shape

(12, 15)